In [1]:
import pandas as pd
import numpy as np
from sklearn import svm
from feature_extractor import featuresFromDataFrame
from tools import random_sample
import igraph


train_df = pd.read_csv("training_set.txt", sep=" ",header=None)
train_df.columns = ["source","target","label"]
train_df = random_sample(train_df)

test_df = pd.read_csv("testing_set.txt", sep=" ",header=None)
test_df.columns = ["source","target"]
#test_df = random_sample(test_df,prop = 1)

node_information_df = pd.read_csv("node_information.csv", sep=",", header=None)
node_information_df.columns = ["ID","year","title","authors","journalName","abstract"]
node_information_df = node_information_df.reset_index().set_index("ID")
node_information_df["authors"].fillna("", inplace=True)

In [2]:
train_df.shape
train_df.head()
test_df.head()
node_information_df.head()

(30775, 3)

,source,target,label
268143,9509029,9211013,0
145983,9912263,9510060,1
363757,202018,9804168,0
138062,9906230,9908026,0
583119,9710154,9611061,0


,source,target
0,9807076,9807139
1,109162,1182
2,9702187,9510135
3,111048,110115
4,9910176,9410073


,index,year,title,authors,journalName,abstract
ID,,,,,,
1001,0,2000,compactification geometry and duality,Paul S. Aspinwall,NaN,these are notes based on lectures given at tas...
1002,1,2000,domain walls and massive gauged supergravity p...,"M. Cvetic, H. Lu, C.N. Pope",Class.Quant.Grav.,we point out that massive gauged supergravity ...
1003,2,2000,comment on metric fluctuations in brane worlds,"Y.S. Myung, Gungwon Kang",NaN,recently ivanov and volovich hep-th 9912242 cl...
1004,3,2000,moving mirrors and thermodynamic paradoxes,Adam D. Helfer,Phys.Rev.,quantum fields responding to moving mirrors ha...
1005,4,2000,bundles of chiral blocks and boundary conditio...,"J. Fuchs, C. Schweigert",NaN,proceedings of lie iii clausthal july 1999 var...


In [6]:
training_features = featuresFromDataFrame(train_df,node_information_df)
testing_features = featuresFromDataFrame(test_df,node_information_df)
labels = train_df["label"].values

In [2]:
from tools import build_graph
g = build_graph()

In [3]:
from feature_extractor import InOutDegree
train_df = InOutDegree(train_df, g)
test_df = InOutDegree(test_df, g)

In [7]:
train_df.to_csv('tmp/train_df')
test_df.to_csv('tmp/test_df')

In [12]:
training_features = np.concatenate((training_features, train_df[['target_in_degree', 'target_out_degree']].values), 
                                   axis=1)
testing_features = np.concatenate((testing_features, test_df[['target_in_degree', 'target_out_degree']].values), 
                                   axis=1)

In [19]:
classifier = svm.LinearSVC()
classifier.fit(training_features, labels)
labels_pred = classifier.predict(testing_features)

prediction_df = pd.DataFrame(columns=["id","category"], dtype=int)
prediction_df["id"] = range(len(labels_pred))
prediction_df["category"] = labels_pred

prediction_df.to_csv("submissions/improved_predictionsinoutdegree.csv", index=None)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)